# Jack’s Car Rental

Jack manages two locations for a nationwide car
rental company. Each day, some number of customers arrive at each location to rent cars.
If Jack has a car available, he rents it out and is credited \$10 by the national company.
If he is out of cars at that location, then the business is lost. Cars become available for
renting the day after they are returned. To help ensure that cars are available where
they are needed, Jack can move them between the two locations overnight, at a cost of
\$2 per car moved. We assume that the number of cars requested and returned at each
location are Poisson random variables, meaning that the probability that the number is
n is $\frac{\lambda^n}{n!}e^{-\lambda}$, where $n$ is the expected number. Suppose $\lambda$ is 3 and 4 for rental requests at
the first and second locations and 3 and 2 for returns. One of Jack’s employees at the first location
rides a bus home each night and lives near the second location. She is happy to shuttle
one car to the second location for free. Each additional car still costs \$2, as do all cars
moved in the other direction. In addition, Jack has limited parking space at each location.
If more than 10 cars are kept overnight at a location (after any moving of cars), then an
additional cost of \$4 must be incurred to use a second parking lot (independent of how
many cars are kept there). To simplify the problem slightly,
we assume that there can be no more than 20 cars at each location (any additional cars
are returned to the nationwide company, and thus disappear from the problem) and a
maximum of five cars can be moved from one location to the other in one night. We take
the discount rate to be $\gamma = 0.9$ and formulate this as a continuing finite MDP, where
the time steps are days, the state is the number of cars at each location at the end of
the day, and the actions are the net numbers of cars moved between the two locations
overnight.

In [ ]:
import numpy as np
import math
from IPython import display
import matplotlib.pyplot as plt
import seaborn as sns
np.set_printoptions(precision=0, suppress=True, linewidth=150)

In [ ]:
# parameters
GAMMA = 0.9 # discount rate
LAMBDA_RENTALS_A = 3
LAMBDA_RENTALS_B = 4
LAMBDA_RETURNS_A = 3
LAMBDA_RETURNS_B = 2
POISSON_LIMIT = 11 # for any n over this, the probability is truncated to 0

In [ ]:
# initalize value and policy tables
v_estimations = np.zeros((21,21))
policy = np.zeros((21,21), dtype=int)

In [ ]:
poisson = {}

# calculating poisson values for efficiency
for l in [LAMBDA_RENTALS_A, LAMBDA_RENTALS_B, LAMBDA_RETURNS_A, LAMBDA_RETURNS_B]:
    for n in range(POISSON_LIMIT):
        poisson[(l, n)] = (np.power(l, n) / math.factorial(n)) * np.exp(-l)

In [ ]:
# calculate the expected value of an action in a given state
def action_value(state, action):
    N_CARS_A = state[0] - action
    N_CARS_B = state[1] + action

    # if the action is illegal in the state, we simply return -infinity
    if (N_CARS_A < 0 or N_CARS_B < 0 or N_CARS_A > 20 or N_CARS_B > 20):
        return -np.inf

    sum_reward = 0
    action_reward = abs(action) * -2

    # one car can be taken from a to b for free
    if action > 0:
        action_reward += 2
    
    # keeping more than 10 cars at a location has a cost of $4
    if N_CARS_A > 10:
        action_reward -= 4
    
    if N_CARS_B > 10:
        action_reward -= 4

    for n_rentals_a in range(POISSON_LIMIT):
        for n_rentals_b in range(POISSON_LIMIT):

            # number of rentals should not be higher than the number of available cars
            real_rentals_a = min(n_rentals_a, N_CARS_A)
            real_rentals_b = min(n_rentals_b, N_CARS_B)

            for n_returns_a in range(POISSON_LIMIT):
                for n_returns_b in range(POISSON_LIMIT):

                    # number of returns should not be higher than the max number of cars
                    n_cars_a = min(N_CARS_A - real_rentals_a + n_returns_a, 20)
                    n_cars_b = min(N_CARS_B - real_rentals_b + n_returns_b, 20)
                    new_state = (n_cars_a, n_cars_b)
                    
                    # joint probability of the rentals and returns happening at both places
                    prob_rentals = poisson[(LAMBDA_RENTALS_A, n_rentals_a)] * poisson[(LAMBDA_RENTALS_B, n_rentals_b)]
                    prob_returns = poisson[(LAMBDA_RETURNS_A, n_returns_a)] * poisson[(LAMBDA_RETURNS_B, n_returns_b)]
                    prob = prob_returns * prob_rentals

                    # add the expected reward to the sum
                    rental_reward = 10 * (real_rentals_a + real_rentals_b)
                    reward = action_reward + rental_reward
                    sum_reward += prob * (reward + GAMMA * v_estimations[new_state])

    # this is the expected reward after taking the action and following the current policy after
    return sum_reward

In [ ]:
# in-place policy evaluation
def policy_evaluation(theta=0.01):
    delta = theta

    while delta >= theta:
        delta = 0
        for n_cars_a in range(21):
            for n_cars_b in range(21):
                state = (n_cars_a, n_cars_b)

                old_value = v_estimations[state]
                action = policy[state]

                v_estimations[state] = action_value(state, action)
                delta = max(delta, abs(old_value - v_estimations[state]))
        
        print(f"delta={delta}")

In [ ]:
def policy_improvement():
    print("Improving policy...")
    policy_stable = True

    for n_cars_a in range(21):
        for n_cars_b in range(21):
            state = (n_cars_a, n_cars_b)
            old_action = policy[state]

            action_values = {}

            # calculate the value of taking each action
            for action in range(-5,6):
                action_values[action] = action_value(state, action)
 
            # keep a list of best actions, so if more equi-best actions are present,
            # and the new policy selects a different one than the old, policy_stable
            # is still True
            best_actions = []
            best_value = -np.inf

            for action, value in action_values.items():
                if value > best_value:
                    best_actions = [action]
                    best_value = value
                elif value == best_value:
                    best_actions.append(action)
            
            policy[state] = best_actions[0]
            policy_stable = policy_stable and old_action in best_actions

    return policy_stable

In [ ]:
# iterate until optimal policy is found
policy_stable = False
i = 1

while not policy_stable:
    display.clear_output()
    print(f"*** Iteration #{i} ***")

    policy_evaluation()
    policy_stable = policy_improvement()
    i += 1

In [ ]:
# print value table
print(v_estimations)

In [ ]:
# print optimal policy
print(policy)

In [ ]:
plt.figure(figsize=(16, 9))
    
sns.heatmap(np.flip(policy, axis=0), cmap='viridis',
            linecolor='white', linewidths=0.05, square=True,
            yticklabels=np.arange(21 - 1, -1, -1))

plt.title(f'Values of the optimal policy')
plt.ylabel('Cars at location 1')
plt.xlabel('Cars at location 2')

plt.show()